In [ ]:
# install.packages("remotes")
# library(remotes)
if (!require("BiocManager", quietly = TRUE))
    install.packages("BiocManager")
# BiocManager::install(version = "3.18")
BiocManager::install("GSVA")

In [ ]:
library(Matrix)
library(GSVA)
library(BiocParallel)

In [ ]:
sparse_matrix <- readMM(snakemake@input[["sparse_matrix"]])
sparse_matrix_c <- as(sparse_matrix, "CsparseMatrix")
sparse_matrix_c

In [ ]:
colnames <- read.csv(snakemake@input[["colnames"]], header = TRUE)[, 1]
colnames

In [ ]:
rownames <- read.csv(snakemake@input[["rownames"]], header = TRUE)[, 1]
rownames

In [ ]:
colnames(sparse_matrix_c) <- colnames
rownames(sparse_matrix_c) <- rownames

In [ ]:
# Load the GMT file for gene sets
# The documentation does not provide a direct function to read GMT files,
# but it suggests using a list where each element is a gene set.
# Here, we use the GSEABase package to read the GMT file.
library(GSEABase)
gene_sets <- getGmt(snakemake@input[["geneset"]])

In [ ]:
gene_sets

In [ ]:
# Set up parallel processing
bpparam <- MulticoreParam(workers = 124, progressbar = TRUE)

gsva_results <- gsva(ssgseaParam(sparse_matrix_c, gene_sets), BPPARAM=bpparam)  # relies on https://github.com/rcastelo/GSVA/issues/113

In [ ]:
gsva_results

In [ ]:
write.csv(gsva_results, file = snakemake@output[["gsva_csv"]], row.names = TRUE)
